<a href="https://www.kaggle.com/code/avinashkushawaha/healthcare?scriptVersionId=282755301" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -q google-genai

from google import genai
import json
import re
import kaggle_secrets

# Load Gemini API key from Kaggle Secrets
GEMINI_API_KEY = kaggle_secrets.UserSecretsClient().get_secret("GEMINI_API_KEY")

# Initialize client
client = genai.Client(api_key=GEMINI_API_KEY)

print("API Loaded Successfully")


API Loaded Successfully


In [2]:
def ask_gemini(prompt, model="gemini-2.5-flash-lite"):
    try:
        response = client.models.generate_content(
            model=model,
            contents=prompt
        )
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"


In [3]:
def medical_ai(user_text):
    
    # STEP 1: Extract symptoms
    symptom_prompt = f"""
    Extract only medical symptoms from this text:
    "{user_text}"
    Return them as a Python list with lowercase names.
    Do not add extra text.
    """
    raw = ask_gemini(symptom_prompt)
    try:
        symptoms = eval(raw)
    except:
        symptoms = []
    
    # STEP 2: Disease Prediction
    disease_prompt = f"""
    Based on symptoms {symptoms}, predict possible illnesses.
    Give 1–3 possibilities max.
    Keep them short, like: ["viral fever", "flu"].
    """
    try:
        diseases = eval(ask_gemini(disease_prompt))
    except:
        diseases = ["Not enough data"]
    
    # STEP 3: Risk Level
    risk_prompt = f"""
    Symptoms: {symptoms}
    Determine a risk level: Low, Medium, or High.
    Rules:
    - breathlessness, chest pain → High
    - fever + cough + fatigue → Medium
    - mild symptoms → Low
    
    Output only one word: "Low", "Medium", or "High".
    """
    risk = ask_gemini(risk_prompt).replace("\n","")
    
    # STEP 4: Doctor Guidance
    doctor_prompt = f"""
    Based on symptoms {symptoms} and risk "{risk}", 
    write:
    - when they must see a doctor
    - emergency warning signs
    
    Keep it short.
    """
    doctor_advice = ask_gemini(doctor_prompt)
    
    # STEP 5: Home Remedies
    remedies_prompt = f"""
    Provide safe home remedies for symptoms {symptoms}.
    No dangerous or prescription drugs.
    """
    remedies = ask_gemini(remedies_prompt)
    
    # STEP 6: OTC Medicines (Safe Only)
    otc_prompt = f"""
    Based on symptoms {symptoms}, give safe OTC options.
    Rules:
    - paracetamol for fever/pain
    - ORS for dehydration
    - cough lozenges for sore throat
    - No antibiotics
    - No prescription drugs
    """
    otc = ask_gemini(otc_prompt)

    # FINAL STRUCTURED OUTPUT
    return {
        "symptoms_detected": symptoms,
        "possible_diseases": diseases,
        "risk_level": risk,
        "doctor_guidance": doctor_advice,
        "home_remedies": remedies,
        "safe_otc": otc
    }


In [4]:
user_input = "I have fever for two days, slight cough, and I feel very tired."
medical_ai(user_input)


{'symptoms_detected': ['fever', 'cough', 'tiredness'],
 'possible_diseases': ['flu', 'common cold', 'viral infection'],
 'risk_level': 'Medium',
 'doctor_guidance': '**When to see a doctor:**\n\n*   If symptoms persist or worsen after a few days.\n\n**Emergency warning signs:**\n\n*   Difficulty breathing\n*   Chest pain\n*   High fever unresponsive to medication',
 'home_remedies': "Here are some safe home remedies for fever, cough, and tiredness. Remember, these are for symptom relief and not cures. If your symptoms are severe, persist for more than a few days, or you have underlying health conditions, it's always best to consult a doctor.\n\n## Home Remedies for Fever:\n\nFever is your body's way of fighting infection. The goal with these remedies is to make you more comfortable and help your body do its work.\n\n*   **Rest:** This is paramount. Your body needs energy to fight off whatever is causing the fever. Stay in bed or on the couch as much as possible.\n*   **Hydration:** Dri

In [5]:
# Test Module for Medical AI
test_cases = [
    "I have fever, body pain and headache since morning.",
    "My throat is hurting and I have a dry cough.",
    "I feel weakness, dizziness and no appetite.",
    "Severe chest pain and shortness of breath.",
    "Runny nose and mild cough for two days."
]

for i, case in enumerate(test_cases, 1):
    print(f"\n--- TEST {i} ---")
    print("Input:", case)
    output = medical_ai(case)
    print("Output:", output)



--- TEST 1 ---
Input: I have fever, body pain and headache since morning.
Output: {'symptoms_detected': ['fever', 'body pain', 'headache'], 'possible_diseases': ['viral fever', 'flu', 'common cold'], 'risk_level': 'Low', 'doctor_guidance': "**See a Doctor:**\n\n*   If symptoms worsen or don't improve after a few days.\n*   If you have underlying health conditions.\n\n**Emergency Warning Signs:**\n\n*   Severe headache or stiff neck\n*   Difficulty breathing\n*   Chest pain\n*   Confusion or lethargy\n*   Rash that doesn't fade when pressed", 'home_remedies': "It's important to remember that while these home remedies can help alleviate symptoms, they are not a substitute for professional medical advice. If your fever is high, persistent, or accompanied by other concerning symptoms, please consult a doctor.\n\nHere are some safe home remedies for fever, body pain, and headache:\n\n**For Fever:**\n\n*   **Hydration is Key:**\n    *   **Water:** Drink plenty of plain water throughout the 

In [6]:
# --- TEST 5 ---
input_text = "I have vomiting and stomach cramps since last night."
print("--- TEST 5 ---")
print("Input:", input_text)
print("Output:", medical_ai(input_text))


--- TEST 5 ---
Input: I have vomiting and stomach cramps since last night.
Output: {'symptoms_detected': [], 'possible_diseases': ['Not enough data'], 'risk_level': "Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15, model: gemini-2.5-flash-lite\\nPlease retry in 35.784087009s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.

In [7]:
def reminder_agent(analysis_output):
    """
    Generates short, AI-agent style reminders for:
    - Medication timing
    - Doctor visit timing
    """

    # --- Medication Reminder ---
    med_prompt = f"""
    Symptoms: {analysis_output.get("symptoms_detected")}
    OTC advice: {analysis_output.get("safe_otc")}

    Create a VERY SHORT medication reminder.
    Rules:
    - If no OTC medicine or only 'paracetamol' → say "Take only if needed"
    - If medicine exists → include simple timing (every 6–8 hours)
    - Output format: {{"medicine": "...", "timing": "..."}}
    - Keep it super short.
    """
    med = ask_gemini(med_prompt)

    # --- Doctor Visit Reminder ---
    doc_prompt = f"""
    Risk: {analysis_output.get("risk_level")}
    Doctor guidance: {analysis_output.get("doctor_guidance")}

    Write a short reminder for when the patient should see a doctor.
    Output 1 short sentence only.
    """
    doc = ask_gemini(doc_prompt)

    return {
        "medication_reminder": med,
        "doctor_visit_reminder": doc
    }


In [8]:
result = medical_ai("I have fever and body pain")
reminders = reminder_agent(result)
result, reminders


({'symptoms_detected': [],
  'possible_diseases': ['Not enough data'],
  'risk_level': "Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15, model: gemini-2.5-flash-lite\\nPlease retry in 34.786382924s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinut

In [9]:
def full_health_agent(user_text):
    analysis = medical_ai(user_text)
    reminders = reminder_agent(analysis)

    # Merge in one clean JSON
    return {
        **analysis,
        "medication_reminder": reminders.get("medication_reminder"),
        "doctor_visit_reminder": reminders.get("doctor_visit_reminder")
    }


In [10]:
full_health_agent("I have fever and body pain")


{'symptoms_detected': [],
 'possible_diseases': ['Not enough data'],
 'risk_level': "Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15, model: gemini-2.5-flash-lite\\nPlease retry in 33.311709497s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePe

In [11]:
def short_health_response(data):
    return {
        "symptoms_detected": data.get("symptoms_detected"),

        "possible_diseases": data.get("possible_diseases"),

        "risk_level": data.get("risk_level"),

        "doctor_guidance": [
            "See a doctor if symptoms worsen or don't improve in 2–3 days.",
            "Go immediately if you have breathing issues, confusion, stiff neck, or severe headache."
        ],

        "home_remedies": [
            "Rest well and stay hydrated.",
            "Use paracetamol for fever/body pain.",
            "Use warm compress for body aches.",
            "Drink warm herbal tea (ginger/chamomile).",
            "Use light clothing and keep room ventilated."
        ],

        "safe_otc": [
            "Paracetamol for fever/body pain.",
            "ORS for dehydration.",
            "Cough lozenges if throat irritation."
        ],

        "medication_reminder": data.get("medication_reminder"),

        "doctor_visit_reminder": data.get("doctor_visit_reminder")
    }


**Here is the final agent which will give user whole details about his desease and also remind him to take medicine and see doctor**

In [12]:
raw = full_health_agent("I have fever and body pain")
short_health_response(raw)


{'symptoms_detected': [],
 'possible_diseases': ['Not enough data'],
 'risk_level': "Error: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \\n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 15, model: gemini-2.5-flash-lite\\nPlease retry in 32.161813714s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePe